# Evaluation of Cross Validation

In [2]:
import sys
sys.path.append('../python/')
from tqdm import tqdm
from glob import glob
import pandas as pd
from parametrized_bootstrapping_model import ParametrizedBootstrappingModel, ReturnAlways1Model, ReturnAlways0Model
from result_analysis_utils import load_ground_truth_data, load_evaluations, run_cross_validation, load_cross_validation_results
SEARCH_SPACE= [0, 1, 2] + list(range(5,96, 5)) + [98, 99, 100]


In [3]:
if 'eval_df' not in locals():
    eval_df = load_evaluations(tqdm(glob('../resources/eval/trec-system-runs/trec13/*.jsonl')))

eval_df.head(2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1637/1637 [01:14<00:00, 22.03it/s]


,,"(depth-10-complete, residual-ndcg@20-min)","(depth-10-complete, residual-ndcg@20-max)","(depth-10-incomplete, residual-ndcg@20-min)","(depth-10-incomplete, residual-ndcg@20-max)","(depth-20-complete, residual-ndcg@20-min)","(depth-20-complete, residual-ndcg@20-max)","(depth-20-incomplete, residual-ndcg@20-min)","(depth-20-incomplete, residual-ndcg@20-max)","(depth-10-complete, condensed-ndcg@10)","(depth-10-incomplete, condensed-ndcg@10)",...,"(depth-20-incomplete, condensed-ndcg@20)","(depth-10-complete, rbp@10)","(depth-10-incomplete, rbp@10)","(depth-20-complete, rbp@10)","(depth-20-incomplete, rbp@10)","(depth-10-complete, unjudged@20)","(depth-10-incomplete, unjudged@20)","(depth-20-complete, unjudged@20)","(depth-20-incomplete, unjudged@20)",run
run,,,,,,,,,,,,,,,,,,,,,,
JuruDes,0,"{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.4303702292214245, ""302"": 0.643239257...","{""301"": 0.21032448436656867, ""302"": 0.64323925...","{""301"": 0.4777169090993102, ""302"": 0.643239257...","{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.39074916216560884, ""302"": 0.64323925...","{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.4380958420434946, ""302"": 0.643239257...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.12987501165334076, ""302"": 0.77215962...",...,"{""301"": 0.39552125507054403, ""302"": 0.64323925...","{""301"": 0.19999999999999993, ""302"": 0.72143923...","{""301"": 0.0, ""302"": 0.7214392319999998, ""303"":...","{""301"": 0.19999999999999993, ""302"": 0.72143923...","{""301"": 0.19999999999999993, ""302"": 0.72143923...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.05, ""304"": 0...","{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....",JuruDes
JuruDesAggr,0,"{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.3661284907500531, ""302"": 0.643239257...","{""301"": 0.27482361468952843, ""302"": 0.64323925...","{""301"": 0.4088868032650847, ""302"": 0.643239257...","{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.3661284907500531, ""302"": 0.643239257...","{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.4088868032650847, ""302"": 0.643239257...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.2180164145324244, ""302"": 0.772159625...",...,"{""301"": 0.41473558366350255, ""302"": 0.64323925...","{""301"": 0.19988479999999997, ""302"": 0.72143923...","{""301"": 0.1179648, ""302"": 0.7214392319999998, ...","{""301"": 0.19988479999999997, ""302"": 0.72143923...","{""301"": 0.19988479999999997, ""302"": 0.72143923...","{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.15, ""302"": 0.0, ""303"": 0.05, ""304"": ...","{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....",JuruDesAggr


In [56]:
print(eval_df[eval_df['run'] == eval_df.iloc[98]['run']].iloc[0].to_dict()[('depth-10-incomplete', 'bs-p-1000-ndcg@10-ndcg@10')])

nan


In [53]:
load_ground_truth_data(
            df=eval_df[eval_df['run'] == eval_df.iloc[98]['run']],
            ground_truth_measure='ndcg',
            depth=10,
            input_measure='bs-p-1000-ndcg@10-ndcg@10',
            random_state=3
        )

TypeError: the JSON object must be str, bytes or bytearray, not float

In [4]:
def run_cross_validation_on_runs(runs):
    ret = []
    for i in tqdm(list(range(runs))):
        ground_truth_data = load_ground_truth_data(
            df=eval_df[eval_df['run'] == eval_df.iloc[i]['run']],
            ground_truth_measure='ndcg',
            depth=10,
            input_measure='bs-p-1000-ndcg@10-ndcg@10',
            random_state=3
        )
        
        models = [ParametrizedBootstrappingModel(i, SEARCH_SPACE) for i in ['rmse[0.6,1]', 'rmse[0.8,1]', 'rmse', 'rmse[1,2]', 'rmse[1,3]', 'rmse[0.1,5]', 'rmse[0.1,10]', 'rmse[0.1,100]', 'rmse[0,1]']] + [ReturnAlways1Model(), ReturnAlways0Model()]

        for model in models:
            tmp = run_cross_validation(ground_truth_data, model)
            
            ret += [tmp]
            
            tmp = tmp.to_json(lines=True, orient='records')
            with open('tmp-results.jsonl', 'a+') as f:
                f.write(tmp)

    return pd.concat(ret)

!rm -f tmp-results.jsonl
results = run_cross_validation_on_runs(95)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [40:09<00:00, 25.36s/it]


In [5]:
results

,run,query,x,y,measures,split,y_prediction,model
0,JuruDes,301,"[0, 0.07336392209936005, 0.22009176629808017, ...",0.220092,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.000000,"pbs-rmse[0.6,1]"
1,JuruDes,302,"[0.7721596254947202, 0.7721596254947202, 0.772...",0.772160,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.772160,"pbs-rmse[0.6,1]"
2,JuruDes,303,"[0.08747171410140821, 0.08747171410140821, 0.0...",0.087472,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.087472,"pbs-rmse[0.6,1]"
3,JuruDes,308,"[0.7541985434892589, 0.787702056960637, 0.7541...",0.636682,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.636682,"pbs-rmse[0.6,1]"
4,JuruDes,310,"[0.6527851779961892, 0.6527851779961892, 0.561...",0.561693,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.561693,"pbs-rmse[0.6,1]"
...,...,...,...,...,...,...,...,...
240,uogRobLWR5,685,"[0.5238203798407154, 0.5238203798407154, 0.523...",0.523820,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
241,uogRobLWR5,686,"[0.4484668750013158, 0.4484668750013158, 0.448...",0.448467,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
242,uogRobLWR5,693,"[0.20959418859680015, 0.20959418859680015, 0.2...",0.165195,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
243,uogRobLWR5,696,"[0.7058075148678525, 0.7058075148678525, 0.705...",0.705808,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0


In [4]:
import pandas as pd
pd.read_json('tmp-results.jsonl', lines=True)

,run,query,x,y,measures,split,y_prediction,model
0,JuruDes,301,"[0, 0.0733639221, 0.22009176630000002, 0.29345...",0.220092,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.073364,"pbs-rmse[0.6,1]"
1,JuruDes,302,"[0.7721596255000001, 0.7721596255000001, 0.772...",0.772160,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.772160,"pbs-rmse[0.6,1]"
2,JuruDes,303,"[0.0874717141, 0.0874717141, 0.0874717141, 0.0...",0.087472,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.087472,"pbs-rmse[0.6,1]"
3,JuruDes,308,"[0.7541985435, 0.7877020570000001, 0.754198543...",0.636682,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.754199,"pbs-rmse[0.6,1]"
4,JuruDes,310,"[0.652785178, 0.652785178, 0.5616927748, 0.652...",0.561693,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0,0.652785,"pbs-rmse[0.6,1]"
...,...,...,...,...,...,...,...,...
156823,uogRobLWR5,685,"[0.5238203798000001, 0.5238203798000001, 0.523...",0.523820,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
156824,uogRobLWR5,686,"[0.44846687500000004, 0.44846687500000004, 0.4...",0.448467,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
156825,uogRobLWR5,693,"[0.2095941886, 0.2095941886, 0.2909617008, 0.2...",0.165195,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0
156826,uogRobLWR5,696,"[0.7058075149, 0.7058075149, 0.7058075149, 0.7...",0.705808,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1,0.000000,always-0


In [1]:
!cp tmp-results.jsonl tmp-results-old.jsonl

In [35]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse": 0.04321960071680512, "quantile": 5, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse": 0.03732903541620089, "quantile": 1, "search_space_size": 25}'],
      dtype=object)

In [36]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse[0,1]', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse[0,1]": 0.0, "quantile": 99, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse[0,1]": 0.0, "quantile": 95, "search_space_size": 25}'],
      dtype=object)

In [37]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse[1,0]', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse[1,0]": 0.0, "quantile": 0, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse[1,0]": 0.0, "quantile": 1, "search_space_size": 25}'],
      dtype=object)

In [38]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse[1,2]', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse[1,2]": 0.09585034291582523, "quantile": 25, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse[1,2]": 0.07465807083240178, "quantile": 1, "search_space_size": 25}'],
      dtype=object)

In [39]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse[1,3]', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse[1,3]": 0.12057778130943755, "quantile": 45, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse[1,3]": 0.09658690063973052, "quantile": 55, "search_space_size": 25}'],
      dtype=object)

In [41]:
tmp = run_cross_validation(ground_truth_data, ParametrizedBootstrappingModel('rmse[0.6,1]', SEARCH_SPACE))
tmp['model'].unique()

array(['{"model": "ParametrizedBootstrappingModel", "rmse[0.6,1]": 0.04884882010933723, "quantile": 1, "search_space_size": 25}',
       '{"model": "ParametrizedBootstrappingModel", "rmse[0.6,1]": 0.03732903541620089, "quantile": 1, "search_space_size": 25}'],
      dtype=object)